# Парсинг PDF → Markdown (Docling)

В этом ноутбуке выполняем конвертацию исходных PDF (PHB/DMG) в Markdown при помощи Docling.

## Параметры обработки

- **Вход**: `data/raw/*.pdf` — исходные PDF-файлы книг правил D&D
- **Выход**: `data/processed/md/{PHB,DMG}.md` — конвертированные Markdown файлы
- **OCR**: включён с языковыми подсказками (`rus`), можно добавить `eng` для улучшения распознавания многоязычных фрагментов
- **Нормализация**: применяется базовая нормализация (исправление переносов, удаление лишних пробелов)

## Почему Markdown?

Мы используем Markdown как основной формат, так как он:
- Лучше сохраняет структуру документа (заголовки, списки, таблицы)
- Сохраняет подписи к изображениям
- Удобен для дальнейшей обработки и секционирования
- Легко читается человеком при необходимости проверки

**Примечание**: Названия книг (PHB, DMG) определяются автоматически из имён PDF-файлов.


In [ ]:
# Импорт и запуск пайплайна Docling для конвертации PDF → Markdown
import sys, os
from pathlib import Path


def _detect_repo_root() -> Path:
    """
    Ищет корень репозитория, поднимаясь по директориям вверх.
    Корень определяется наличием папок 'src' и 'data'.
    """
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start


try:
    # Сработает, если код запускается как скрипт, а не ноутбук
    repo_root = Path(__file__).resolve().parent.parent
except NameError:
    # В ноутбуке __file__ нет — ищем корень по структуре проекта
    repo_root = _detect_repo_root()


# Добавляем src/ в PYTHONPATH, чтобы импортировать пакет dnd_rag
sys.path.append(str(repo_root / 'src'))

# Импортируем функцию пайплайна для парсинга PDF
from dnd_rag.core.pipelines import parse_docs_pipeline  # type: ignore

# Указываем пути к директориям
RAW_DIR = repo_root / 'data' / 'raw'  # Папка с исходными PDF файлами
OUT_MD_DIR = repo_root / 'data' / 'processed' / 'md'  # Папка для результатов Markdown

# Запускаем конвертацию всех PDF файлов в Markdown
# ocr=True - включаем оптическое распознавание текста для сканов и изображений
# ocr_langs - указываем языки для OCR (русский). Можно добавить "eng" для английского
produced = parse_docs_pipeline(
    RAW_DIR,
    OUT_MD_DIR,
    ocr=True,
    ocr_langs=("rus",),  # Языковые подсказки для лучшего распознавания
)

# Выводим список созданных файлов
produced


## Проверка результата

После выполнения ячейки выше должны появиться файлы:

- `data/processed/md/PHB.md` — Player's Handbook в формате Markdown
- `data/processed/md/DMG.md` — Dungeon Master's Guide в формате Markdown

### Что проверить в результатах:

- ✅ Заголовки корректны и соответствуют структуре книги
- ✅ Таблицы читаемы и сохранили форматирование
- ✅ Подписи к изображениям включены в текст (если были в оригинале)
- ✅ Текст разбит на параграфы
- ✅ Буквицы и специальные символы распознаны (могут требовать дополнительной очистки в следующем шаге)

**Следующий шаг**: Глубокая нормализация Markdown для исправления OCR-артефактов (ноутбук `02_md_processing.ipynb`).
